In [13]:
pip install langserve

Note: you may need to restart the kernel to use updated packages.


In [17]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
import os
from langchain.chains import LLMChain
from dotenv import load_dotenv
from langserve import add_routes
import uvicorn

# Load environment variables
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Initialize FastAPI app
mb_app = FastAPI(title="Langchain Server", 
                 version="1.0",
                 description="A simple API Server")

# Define models and prompts
model = ChatOpenAI()  # You were correctly initializing this before
llm = Ollama(model="llama2")

prompt1 = ChatPromptTemplate.from_template("Give me information about {topic} with 150 words")
prompt2 = ChatPromptTemplate.from_template("Give me a song about {topic} with 100 words")

# Create LLM chains (which are Runnable instances)
llm_chain1 = LLMChain(llm=model, prompt=prompt1)
llm_chain2 = LLMChain(llm=llm, prompt=prompt2)

# Add routes for information and song using the LLMChains
add_routes(mb_app, llm_chain1, path="/information")
add_routes(mb_app, llm_chain2, path="/song")

# Run the app
if __name__ == "__main__":
    uvicorn.run(mb_app, host="localhost", port=8000)
